# install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab Notebooks/master_thesis
!pip install transformers
!pip install pytorch-lightning
#!pip install git+https://github.com/facebookresearch/text-simplification-evaluation.git
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.optim as optim
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
#from torch.utils.data.sampler import BatchSampler
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import tqdm
import itertools
import pickle
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore, spearmanr, pearsonr

# margin ranking model

In [ ]:
from ast import Add
from cProfile import label
import os
from random import shuffle
#import hydra
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
#from omegaconf import DictConfig
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional import accuracy, auroc
from transformers import BertModel, BertTokenizer
#from tseval.feature_extraction import get_compression_ratio, get_wordrank_score
import spacy
import tqdm
#import nltk
#nltk.doenload('punkt')


class AugmentedDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int,
        orig_column_name: str,
        simp_column_name: str,
        sent_id_column_name: str,
        sys_name_column_name: str,
    ):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.orig_column_name = orig_column_name
        self.simp_column_name = simp_column_name
        self.sent_id_column_name = sent_id_column_name
        self.sys_name_column_name = sys_name_column_name

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data_row = self.data.iloc[index]
        orig = data_row[self.orig_column_name]
        simp = data_row[self.simp_column_name]
        sent_id = data_row[self.sent_id_column_name]
        sys_name = data_row[self.sys_name_column_name]

        encoding_orig_orig = self.tokenizer.encode_plus(
            #origin,
            orig,
            orig,
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        encoding_orig_simp = self.tokenizer.encode_plus(
            #origin,
            orig,
            simp,
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        encoding_simp_simp = self.tokenizer.encode_plus(
            #origin,
            simp,
            simp,
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        encoding_simp_orig = self.tokenizer.encode_plus(
            #origin,
            simp,
            orig,
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return dict(
            orig_orig=dict(
                input_ids=encoding_orig_orig["input_ids"].flatten(),
                attention_mask=encoding_orig_orig["attention_mask"].flatten(),
                #added_features=orig_orig_added_features_tensor.flatten(),
            ),
            orig_simp=dict(
                input_ids=encoding_orig_simp["input_ids"].flatten(),
                attention_mask=encoding_orig_simp["attention_mask"].flatten(),
                #added_features=orig_simp_added_features_tensor.flatten(),
                sents=simp
            ),
            simp_simp=dict(
                input_ids=encoding_simp_simp["input_ids"].flatten(),
                attention_mask=encoding_simp_simp["attention_mask"].flatten(),
                #added_features=orig_simp_added_features_tensor.flatten(),
                sents=simp
            ),
            simp_orig=dict(
                input_ids=encoding_simp_orig["input_ids"].flatten(),
                attention_mask=encoding_simp_orig["attention_mask"].flatten(),
                #added_features=orig_simp_added_features_tensor.flatten(),
            ),
            sent_ids=sent_id,
            sys_names=sys_name,
            #labels=torch.tensor(label),
            #case_nums=torch.tensor(case_num),
        )

class CreateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df: pd.DataFrame=None,
        valid_df: pd.DataFrame=None,
        test_df: pd.DataFrame=None,
        batch_size: int=None,
        max_token_len: int=None,
        #origin_column_name: str = 'origin',
        orig_column_name: str = 'orig_sent',
        simp_column_name: str = 'simp_sent',
        sent_id_column_name: str = 'sent_id',
        sys_name_column_name: str = 'sys_name',
        #label_column_name: str = 'label',
        #case_num_column_name: str = 'case_number',
        pretrained_model='bert-base-uncased',
    ):
        super().__init__()
        self.train_df = train_df
        self.valid_df = valid_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        #self.origin_column_name = origin_column_name
        self.orig_column_name = orig_column_name
        self.simp_column_name = simp_column_name
        self.sent_id_column_name = sent_id_column_name
        self.sys_name_column_name = sys_name_column_name
        #self.label_column_name = label_column_name
        #self.case_num_column_name = case_num_column_name
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_model)

    def setup(self, stage):
        if stage == "test":
          self.test_dataset = AugmentedDataset(
              self.test_df,
              self.tokenizer,
              self.max_token_len,
              #self.origin_column_name,
              self.orig_column_name,
              self.simp_column_name,
              self.sent_id_column_name,
              self.sys_name_column_name,
              #self.label_column_name,
              #self.case_num_column_name,
            )

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=os.cpu_count())

class BertRanker(pl.LightningModule):
    def __init__(
        self,
        n_classes: int,
        learning_rate: float,
        pooling_type: str,
        added_feature_num: int,
        pretrained_model='bert-base-uncased',
    ):
        super().__init__()
        self.bert = BertModel.from_pretrained(pretrained_model, output_hidden_states=True, return_dict=True)
        classifier_hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(classifier_hidden_size + added_feature_num, n_classes)

        self.lr = learning_rate
        self.criterion = nn.MarginRankingLoss(margin=1.0)
        self.n_classes = n_classes
        self.pooling_type = pooling_type

        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True
        for param in self.classifier.parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        if self.pooling_type == 'cls':
            cls = output.pooler_output
            #output = torch.cat([added_features.float(), cls], dim=1)
            preds = self.classifier(cls)
        #preds = torch.flatten(preds)
        return preds, output

# prepare eval data

In [ ]:
with open('./src/simplicityDA_silver_test.pickle', 'rb') as f:
    df_simplicityDA = pickle.load(f)

# setting

In [ ]:
max_token_len=512
learning_rate=0.05
pooling_type = 'cls'
added_feature_num = 0
feature_path = 'wo_features/'
metrics_segment_path = 'metrics_segment_c/'

# evaluation (human eval)

In [ ]:
ckpaths = ['exp2/base/100/m=1/epoch=', 'exp2/silver/100/m=1/epoch=']
epoch_lists = [['47', '42', '41', '31', '29', '28', '28 (1)', '22', '21', '19'], ['46', '37', '37 (1)', '33', '33 (1)', '30', '26', '26 (1)', '23', '23 (1)']] #v11_nprm_filtered_simplicityDA
epoch_lists = [['31', '30', '30 (1)', '29', '25', '25 (1)', '24', '23', '23 (1)', '22'], ['18 (1)', '19 (1)', '21 (2)', '18', '19', '25', '17', '20', '21 (1)', '21']] #v11_nprm_filtered_simplicityDA_trial2

for ckpath, epoch_list in zip(ckpaths, epoch_lists):
    df_metrics_segment_DA = pd.DataFrame()
    df_metrics_segment_hg = pd.DataFrame()
    df_metrics_segment_simpDA = pd.DataFrame()
    for i, epoch_num in tqdm.tqdm(enumerate(epoch_list)):
        now_model = 'model_' + str(i)
        model = BertRanker.load_from_checkpoint(
            n_classes=2,
            learning_rate=learning_rate,
            pooling_type=pooling_type,
            added_feature_num = added_feature_num,
            checkpoint_path='./lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/' + feature_path + ckpath + epoch_num + '.ckpt',
        )
        #model = nn.DataParallel(model, device_ids=[2,3])
        device = 'cuda'
        model.to(device)

        data_module = CreateDataModule(
        test_df=df_simplicityDA,
        batch_size=1,
        max_token_len=max_token_len,
        )
        data_module.setup(stage='test')
        test_dataloader = data_module.test_dataloader()
        metrics_DA = []
        sent_ids = []
        sys_names = []
        test_sets = []
        with torch.no_grad():
            print("start prediction of mr model")
            for batch in test_dataloader:
                input_ids=batch["orig_simp"]["input_ids"]
                attention_mask=batch["orig_simp"]["attention_mask"]

                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)

                preds_orig_simp, output = model(input_ids, attention_mask)
                preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)

                input_ids=batch["simp_orig"]["input_ids"]
                attention_mask=batch["simp_orig"]["attention_mask"]

                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)

                preds_simp_orig, output = model(input_ids, attention_mask)
                preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)

                #metrics_DA.append(preds_orig_simp[0].item() - preds_orig_orig[0].item())
                #metrics_DA.append(preds_orig_simp[0][1].item()) # pattern a
                #metrics_DA.append(preds_simp_orig[0][0].item()) # pattern b
                metrics_DA.append((preds_orig_simp[0][1].item() + preds_simp_orig[0][0].item()) / 2) # pattern c
                #metrics_DA.append(min(preds_orig_simp[0][1].item(), preds_simp_orig[0][0].item())) # pattern f
                #metrics_DA.append(max(preds_orig_simp[0][1].item(), preds_simp_orig[0][0].item())) # pattern g
                sent_ids.append(int(batch['sent_ids'][0]))
                sys_names.append(batch['sys_names'][0])
                test_sets.append('asset')

        if len(df_metrics_segment_DA) != 0:
            df_metrics_segment_DA = df_metrics_segment_DA.merge(pd.DataFrame({'sent_id':sent_ids, 'sys_name':sys_names, 'test_set':test_sets, now_model:metrics_DA}), on=['sent_id', 'sys_name', 'test_set'])
        else:
            df_metrics_segment_DA = pd.DataFrame({'sent_id':sent_ids, 'sys_name':sys_names, 'test_set':test_sets, now_model:metrics_DA})

    save_name = feature_path.replace('/', '_') + ckpath[:-7].replace('/', '_')

    df_metrics_segment_DA_save = df_metrics_segment_DA[['sent_id', 'sys_name', 'test_set']].copy()
    df_metrics_segment_DA_save[save_name] = df_metrics_segment_DA.drop(columns=['sent_id', 'sys_name', 'test_set']).mean(axis=1)
    df_metrics_segment_DA_save = df_metrics_segment_DA_save.merge(df_metrics_segment_DA, on=['sent_id', 'sys_name', 'test_set'])
    with open('./lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/' + feature_path + ckpath[:5] + metrics_segment_path + save_name + '_DA'  + '.pickle', 'wb') as f:
        pickle.dump(df_metrics_segment_DA_save, f)

0it [00:00, ?it/s]INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/wo_features/exp2/base/100/m=1/epoch=31.ckpt`


start prediction of mr model


<ipython-input-13-469726bc9c85>:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)
<ipython-input-13-469726bc9c85>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)
1it [00:10, 10.58s/it]INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/wo_features/exp2/base/100/m=1/epoch=30.ckpt`


start prediction of mr model


<ipython-input-13-469726bc9c85>:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)
<ipython-input-13-469726bc9c85>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)
2it [00:20,  9.95s/it]INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/wo_features/exp2/base/100/m=1/epoch=30 (1).ckpt`


...

<ipython-input-13-469726bc9c85>:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)
<ipython-input-13-469726bc9c85>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)
8it [01:17,  9.51s/it]INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/wo_features/exp2/silver/100/m=1/epoch=21 (1).ckpt`


start prediction of mr model


<ipython-input-13-469726bc9c85>:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)
<ipython-input-13-469726bc9c85>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)
9it [01:26,  9.49s/it]INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/wo_features/exp2/silver/100/m=1/epoch=21.ckpt`


start prediction of mr model


<ipython-input-13-469726bc9c85>:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_orig_simp = torch.nn.functional.softmax(preds_orig_simp)
<ipython-input-13-469726bc9c85>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_simp_orig = torch.nn.functional.softmax(preds_simp_orig)
10it [01:37,  9.70s/it]


# correlations

## import

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore, pearsonr, spearmanr, t, kendalltau
from tqdm import tqdm
import pickle


def compute_direct_assessment_correlations(df_human_scores, df_metrics_scores, aspect, segment_id_cols,
                                           use_absolute_values=True):
    df_da_scores = df_human_scores.reset_index()
    cols_of_interest = segment_id_cols + [f"{aspect}_zscore"]
    df_da_scores = df_da_scores[cols_of_interest]
    df_all_scores = pd.merge(left=df_metrics_scores, right=df_da_scores, on=segment_id_cols)

    # Compute correlations metrics vs human scores
    #print("Computing correlations...")
    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]
    correlations_data = []
    for metric in metrics_names:
        corr, p_value = pearsonr(df_all_scores[metric], df_all_scores[f'{aspect}_zscore'])
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr, p_value])
    df_correlations_metrics_human = pd.DataFrame(correlations_data, columns=['metric', 'corr', 'p_value'])
    df_correlations_metrics_human.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Compute correlations metrics vs metrics
    metrics_names = df_correlations_metrics_human['metric'].to_list()
    correlations_data = []
    for _, (metric_a, corr_metric_a, _) in df_correlations_metrics_human.iterrows():
        for _, (metric_b, corr_metric_b, _) in df_correlations_metrics_human.iterrows():
            corr_a_b, pvalue_a_b = pearsonr(df_all_scores[metric_a], df_all_scores[metric_b])
            if use_absolute_values:
                corr_a_b = abs(corr_a_b)
            correlations_data.append([metric_a, corr_metric_a,
                                      metric_b, corr_metric_b,
                                      corr_a_b, pvalue_a_b])
    df_correlations_metric_metric = pd.DataFrame(correlations_data,
                                                 columns=['metric_a', 'corr_metric_a',
                                                          'metric_b', 'corr_metric_b',
                                                          'corr_a_b', 'pvalue_a_b'])

    # Determine if the difference in performance is significant
    #print("Determining if the difference in performance is significant...")
    significance_matrix = []
    winner_status = []
    for metric_a in metrics_names:
        df_correlations = df_correlations_metric_metric[df_correlations_metric_metric['metric_a'] == metric_a]
        is_winner = True
        significance_row = []
        for _, (_, corr_metric_a, metric_b, corr_metric_b, corr_a_b, _) in df_correlations.iterrows():
            p = np.nan
            if (metric_a != metric_b) and (corr_metric_a > corr_metric_b):
                _, p = williams_test(corr_metric_a, corr_metric_b, corr_a_b, len(df_human_scores))
            is_diff_stats_significant = p < 0.05
            if not is_diff_stats_significant:
                # we do not care about the exact values in cases where it's not significant
                p = np.nan
            significance_row.append(p)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_a != metric_b:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations_metrics_human['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations_metrics_human, df_significance

def compute_relative_ranking_correlations(df_human_scores, df_metrics_scores, aspect, segment_id_cols,
                                           use_absolute_values=True):
    df_da_scores = df_human_scores.reset_index()
    cols_of_interest = segment_id_cols + [f"{aspect}_zscore"]
    df_da_scores = df_da_scores[cols_of_interest]
    df_all_scores = pd.merge(left=df_metrics_scores, right=df_da_scores, on=segment_id_cols)

    # Compute correlations metrics vs human scores
    #print("Computing correlations...")
    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]
    correlations_data = []
    for metric in metrics_names:
        corr, p_value = spearmanr(df_all_scores[metric], df_all_scores[f'{aspect}_zscore'])
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr, p_value])
    df_correlations_metrics_human = pd.DataFrame(correlations_data, columns=['metric', 'corr', 'p_value'])
    df_correlations_metrics_human.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Compute correlations metrics vs metrics
    metrics_names = df_correlations_metrics_human['metric'].to_list()
    correlations_data = []
    for _, (metric_a, corr_metric_a, _) in df_correlations_metrics_human.iterrows():
        for _, (metric_b, corr_metric_b, _) in df_correlations_metrics_human.iterrows():
            corr_a_b, pvalue_a_b = spearmanr(df_all_scores[metric_a], df_all_scores[metric_b])
            if use_absolute_values:
                corr_a_b = abs(corr_a_b)
            correlations_data.append([metric_a, corr_metric_a,
                                      metric_b, corr_metric_b,
                                      corr_a_b, pvalue_a_b])
    df_correlations_metric_metric = pd.DataFrame(correlations_data,
                                                 columns=['metric_a', 'corr_metric_a',
                                                          'metric_b', 'corr_metric_b',
                                                          'corr_a_b', 'pvalue_a_b'])

    # Determine if the difference in performance is significant
    #print("Determining if the difference in performance is significant...")
    significance_matrix = []
    winner_status = []
    for metric_a in metrics_names:
        df_correlations = df_correlations_metric_metric[df_correlations_metric_metric['metric_a'] == metric_a]
        is_winner = True
        significance_row = []
        for _, (_, corr_metric_a, metric_b, corr_metric_b, corr_a_b, _) in df_correlations.iterrows():
            p = np.nan
            if (metric_a != metric_b) and (corr_metric_a > corr_metric_b):
                _, p = williams_test(corr_metric_a, corr_metric_b, corr_a_b, len(df_human_scores))
            is_diff_stats_significant = p < 0.05
            if not is_diff_stats_significant:
                # we do not care about the exact values in cases where it's not significant
                p = np.nan
            significance_row.append(p)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_a != metric_b:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations_metrics_human['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations_metrics_human, df_significance


def compute_kendall_tau_correlations(df_human_scores, df_metrics_scores, aspect, segment_id_cols,
                                           use_absolute_values=True):
    df_da_scores = df_human_scores.reset_index()
    cols_of_interest = segment_id_cols + [f"{aspect}_zscore"]
    df_da_scores = df_da_scores[cols_of_interest]
    df_all_scores = pd.merge(left=df_metrics_scores, right=df_da_scores, on=segment_id_cols)

    # Compute correlations metrics vs human scores
    #print("Computing correlations...")
    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]
    correlations_data = []
    for metric in metrics_names:
        corr, p_value = kendalltau(df_all_scores[metric], df_all_scores[f'{aspect}_zscore'])
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr, p_value])
    df_correlations_metrics_human = pd.DataFrame(correlations_data, columns=['metric', 'corr', 'p_value'])
    df_correlations_metrics_human.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Compute correlations metrics vs metrics
    metrics_names = df_correlations_metrics_human['metric'].to_list()
    correlations_data = []
    for _, (metric_a, corr_metric_a, _) in df_correlations_metrics_human.iterrows():
        for _, (metric_b, corr_metric_b, _) in df_correlations_metrics_human.iterrows():
            corr_a_b, pvalue_a_b = kendalltau(df_all_scores[metric_a], df_all_scores[metric_b])
            if use_absolute_values:
                corr_a_b = abs(corr_a_b)
            correlations_data.append([metric_a, corr_metric_a,
                                      metric_b, corr_metric_b,
                                      corr_a_b, pvalue_a_b])
    df_correlations_metric_metric = pd.DataFrame(correlations_data,
                                                 columns=['metric_a', 'corr_metric_a',
                                                          'metric_b', 'corr_metric_b',
                                                          'corr_a_b', 'pvalue_a_b'])

    # Determine if the difference in performance is significant
    #print("Determining if the difference in performance is significant...")
    significance_matrix = []
    winner_status = []
    for metric_a in metrics_names:
        df_correlations = df_correlations_metric_metric[df_correlations_metric_metric['metric_a'] == metric_a]
        is_winner = True
        significance_row = []
        for _, (_, corr_metric_a, metric_b, corr_metric_b, corr_a_b, _) in df_correlations.iterrows():
            p = np.nan
            if (metric_a != metric_b) and (corr_metric_a > corr_metric_b):
                _, p = williams_test(corr_metric_a, corr_metric_b, corr_a_b, len(df_human_scores))
            is_diff_stats_significant = p < 0.05
            if not is_diff_stats_significant:
                # we do not care about the exact values in cases where it's not significant
                p = np.nan
            significance_row.append(p)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_a != metric_b:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations_metrics_human['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations_metrics_human, df_significance


# From https://github.com/inmoonlight/nlp-williams/blob/master/williams.py
def williams_test(r12, r13, r23, n):

    """The Williams test (Evan J. Williams. 1959. Regression Analysis, volume 14. Wiley, New York, USA)
    A test of whether the population correlation r12 equals the population correlation r13.
    Significant: p < 0.05
    Arguments:
        r12 (float): correlation between x1, x2
        r13 (float): correlation between x1, x3
        r23 (float): correlation between x2, x3
        n (int): size of the population
    Returns:
        t (float): Williams test result
        p (float): p-value of t-dist
    """
    assert (r12 >= r13), "r12 should be larger than r13"
    assert (n > 3), "n should be larger than 3"

    K = 1 - r12 ** 2 - r13 ** 2 - r23 ** 2 + 2 * r12 * r13 * r23
    denominator = np.sqrt(
        2 * K * (n - 1) / (n - 3) + (((r12 + r13) ** 2) / 4) * ((1 - r23) ** 3)
    )
    numerator = (r12 - r13) * np.sqrt((n - 1) * (1 + r23))
    p = 1 - t.cdf(numerator / denominator, df=n - 3)  # changed to n-3 on 30/11/14
    return t, p

In [ ]:
def compute_direct_correlations(df_benchmark, df_segment_metrics, aspects, test_sets, segment_id_cols):
    results = {}
    for aspect in aspects:
        smed = df_benchmark['simplicity_zscore'].median()
        shind = df_benchmark['simplicity_zscore'] > smed
        slind = df_benchmark['simplicity_zscore'] <= smed
        fmed = df_benchmark['fluency_zscore'].median()
        fhind = df_benchmark['fluency_zscore'] > fmed
        flind = df_benchmark['fluency_zscore'] <= fmed
        mmed = df_benchmark['meaning_zscore'].median()
        mhind = df_benchmark['meaning_zscore'] > mmed
        mlind = df_benchmark['meaning_zscore'] <= mmed
        fmhind = (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        fmlind = (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        smfhind = (df_benchmark['simplicity_zscore'] > smed) & (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        smflind = (df_benchmark['simplicity_zscore'] <= smed) & (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        df_scores_slow = df_benchmark[slind]
        df_scores_shigh = df_benchmark[shind]
        df_scores_flow = df_benchmark[flind]
        df_scores_fhigh = df_benchmark[fhind]
        df_scores_mlow = df_benchmark[mlind]
        df_scores_mhigh = df_benchmark[mhind]
        df_scores_fmlow = df_benchmark[fmlind]
        df_scores_fmhigh = df_benchmark[fmhind]
        df_scores_sfmlow = df_benchmark[smflind]
        df_scores_sfmhigh = df_benchmark[smfhind]

        print(f"{aspect}: simp_high ({len(df_scores_shigh)}) - simp_low ({len(df_scores_slow)}) - flu_low ({len(df_scores_flow)}) - flu_high ({len(df_scores_fhigh)}) - mean_low ({len(df_scores_mlow)}) - mean_high ({len(df_scores_mhigh)}) - fm_low ({len(df_scores_fmlow)}) - fm_high ({len(df_scores_fmhigh)}) - sfm_low ({len(df_scores_sfmlow)}) - sfm_high ({len(df_scores_sfmhigh)}) - All ({len(df_benchmark)})")
        for quality, df_scores in {'simp_low': df_scores_slow, 'simp_high': df_scores_shigh, 'flu_low': df_scores_flow, 'flu_high': df_scores_fhigh, 'mean_low': df_scores_mlow, 'mean_high': df_scores_mhigh, 'fm_low': df_scores_fmlow, 'fm_high': df_scores_fmhigh, 'sfm_low': df_scores_sfmlow, 'sfm_high': df_scores_sfmhigh, 'all': df_benchmark}.items():
            for test_set in test_sets:
                #print(f"Computing for {quality} scores - {test_set} references")
                df_metrics = df_segment_metrics[df_segment_metrics.test_set==test_set].drop(columns=['test_set'])
                results[(quality, aspect, test_set)] = compute_direct_assessment_correlations(
                    df_scores,
                    df_metrics,
                    aspect,
                    segment_id_cols=segment_id_cols,
                    use_absolute_values=False
                )
                #print()
    return results

def compute_ranking_correlations(df_benchmark, df_segment_metrics, aspects, test_sets, segment_id_cols):
    results = {}
    for aspect in aspects:
        smed = df_benchmark['simplicity_zscore'].median()
        shind = df_benchmark['simplicity_zscore'] > smed
        slind = df_benchmark['simplicity_zscore'] <= smed
        fmed = df_benchmark['fluency_zscore'].median()
        fhind = df_benchmark['fluency_zscore'] > fmed
        flind = df_benchmark['fluency_zscore'] <= fmed
        mmed = df_benchmark['meaning_zscore'].median()
        mhind = df_benchmark['meaning_zscore'] > mmed
        mlind = df_benchmark['meaning_zscore'] <= mmed
        fmhind = (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        fmlind = (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        smfhind = (df_benchmark['simplicity_zscore'] > smed) & (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        smflind = (df_benchmark['simplicity_zscore'] <= smed) & (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        df_scores_slow = df_benchmark[slind]
        df_scores_shigh = df_benchmark[shind]
        df_scores_flow = df_benchmark[flind]
        df_scores_fhigh = df_benchmark[fhind]
        df_scores_mlow = df_benchmark[mlind]
        df_scores_mhigh = df_benchmark[mhind]
        df_scores_fmlow = df_benchmark[fmlind]
        df_scores_fmhigh = df_benchmark[fmhind]
        df_scores_sfmlow = df_benchmark[smflind]
        df_scores_sfmhigh = df_benchmark[smfhind]

        print(f"{aspect}: simp_high ({len(df_scores_shigh)}) - simp_low ({len(df_scores_slow)}) - flu_low ({len(df_scores_flow)}) - flu_high ({len(df_scores_fhigh)}) - mean_low ({len(df_scores_mlow)}) - mean_high ({len(df_scores_mhigh)}) - fm_low ({len(df_scores_fmlow)}) - fm_high ({len(df_scores_fmhigh)}) - sfm_low ({len(df_scores_sfmlow)}) - sfm_high ({len(df_scores_sfmhigh)}) - All ({len(df_benchmark)})")
        for quality, df_scores in {'simp_low': df_scores_slow, 'simp_high': df_scores_shigh, 'flu_low': df_scores_flow, 'flu_high': df_scores_fhigh, 'mean_low': df_scores_mlow, 'mean_high': df_scores_mhigh, 'fm_low': df_scores_fmlow, 'fm_high': df_scores_fmhigh, 'sfm_low': df_scores_sfmlow, 'sfm_high': df_scores_sfmhigh, 'all': df_benchmark}.items():
            for test_set in test_sets:
                #print(f"Computing for {quality} scores - {test_set} references")
                df_metrics = df_segment_metrics[df_segment_metrics.test_set==test_set].drop(columns=['test_set'])
                results[(quality, aspect, test_set)] = compute_relative_ranking_correlations(
                    df_human_scores=df_scores,
                    df_metrics_scores=df_metrics,
                    aspect=aspect,
                    segment_id_cols=segment_id_cols,
                    #sentence_id_cols=['sent_id'],
                    #system_id_cols=['sys_name'],
                    use_absolute_values=False,
                )
                #print()
    return results

def compute_kendall_correlations(df_benchmark, df_segment_metrics, aspects, test_sets, segment_id_cols):
    results = {}
    for aspect in aspects:
        smed = df_benchmark['simplicity_zscore'].median()
        shind = df_benchmark['simplicity_zscore'] > smed
        slind = df_benchmark['simplicity_zscore'] <= smed
        fmed = df_benchmark['fluency_zscore'].median()
        fhind = df_benchmark['fluency_zscore'] > fmed
        flind = df_benchmark['fluency_zscore'] <= fmed
        mmed = df_benchmark['meaning_zscore'].median()
        mhind = df_benchmark['meaning_zscore'] > mmed
        mlind = df_benchmark['meaning_zscore'] <= mmed
        fmhind = (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        fmlind = (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        df_scores_slow = df_benchmark[slind]
        df_scores_shigh = df_benchmark[shind]
        df_scores_flow = df_benchmark[flind]
        df_scores_fhigh = df_benchmark[fhind]
        df_scores_mlow = df_benchmark[mlind]
        df_scores_mhigh = df_benchmark[mhind]
        df_scores_fmlow = df_benchmark[fmlind]
        df_scores_fmhigh = df_benchmark[fmhind]

        print(f"{aspect}: simp_high ({len(df_scores_shigh)}) - simp_low ({len(df_scores_slow)}) - flu_low ({len(df_scores_flow)}) - flu_high ({len(df_scores_fhigh)}) - mean_low ({len(df_scores_mlow)}) - mean_high ({len(df_scores_mhigh)}) - fm_low ({len(df_scores_fmlow)}) - fm_high ({len(df_scores_fmhigh)}) - All ({len(df_benchmark)})")
        for quality, df_scores in {'simp_low': df_scores_slow, 'simp_high': df_scores_shigh, 'flu_low': df_scores_flow, 'flu_high': df_scores_fhigh, 'mean_low': df_scores_mlow, 'mean_high': df_scores_mhigh, 'fm_low': df_scores_fmlow, 'fm_high': df_scores_fmhigh, 'all': df_benchmark}.items():
            for test_set in test_sets:
                #print(f"Computing for {quality} scores - {test_set} references")
                df_metrics = df_segment_metrics[df_segment_metrics.test_set==test_set].drop(columns=['test_set'])
                results[(quality, aspect, test_set)] = compute_kendall_tau_correlations(
                    df_human_scores=df_scores,
                    df_metrics_scores=df_metrics,
                    aspect=aspect,
                    segment_id_cols=segment_id_cols,
                    #sentence_id_cols=['sent_id'],
                    #system_id_cols=['sys_name'],
                    use_absolute_values=False,
                )
                #print()
    return results

In [ ]:
with open('./src/simplicityDA_silver_test.pickle', 'rb') as f:
    df_simplicityDA = pickle.load(f)

## calculation

In [ ]:
feature_path = 'wo_features/'
#metrics_segment_path = 'metrics_segment_a/'
#metrics_segment_path = 'metrics_segment_b/'
metrics_segment_path = 'metrics_segment_c/'
ckpaths = ['exp2/base/100/m=1/epoch=', 'exp2/silver/100/m=1/epoch=']

In [ ]:
df_metrics_segment_DA_all = pd.DataFrame()
ckpath = ckpaths[1] #change here
save_name = feature_path.replace('/', '_') + ckpath[:-7].replace('/', '_')
path_DA = './lightning_logs_folder/v11_nprm_filtered_simplicityDA_trial2/' + feature_path + ckpath[:5] + metrics_segment_path + save_name + '_DA'  + '.pickle'
with open(path_DA, 'rb') as f:
    df_metrics_segment_DA = pickle.load(f)

if len(df_metrics_segment_DA_all) != 0:
    df_metrics_segment_DA_all = df_metrics_segment_DA_all.merge(df_metrics_segment_DA, on=['sent_id', 'sys_name', 'test_set'])
else:
    df_metrics_segment_DA_all = df_metrics_segment_DA

In [ ]:
results_simplicity = compute_direct_correlations(
                        df_simplicityDA,
                        df_metrics_segment_DA_all,
                        aspects=['simplicity'],
                        test_sets=['asset'],
                        segment_id_cols=['sent_id','sys_name']
                    )
print(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"].mean())
print(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"].std())
print(list(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"]))

simplicity: simp_high (60) - simp_low (60) - flu_low (60) - flu_high (60) - mean_low (60) - mean_high (60) - fm_low (43) - fm_high (43) - sfm_low (40) - sfm_high (36) - All (120)
0.6212664875112741
0.025844181946461306
[0.660374080045129, 0.6485903193061069, 0.6369042885180044, 0.6330453032550467, 0.6323196256618717, 0.6203657021822395, 0.6084012163026571, 0.6020280060483245, 0.5926950770378598, 0.5779412567555016]


In [ ]:
results_simplicity = compute_ranking_correlations(
                        df_simplicityDA,
                        df_metrics_segment_DA_all,
                        aspects=['simplicity'],
                        test_sets=['asset'],
                        segment_id_cols=['sent_id','sys_name']
                    )
print(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"].mean())
print(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"].std())
print(list(results_simplicity[('all', 'simplicity', 'asset')][0][results_simplicity[('all', 'simplicity', 'asset')][0]["metric"] != save_name]["corr"]))

simplicity: simp_high (60) - simp_low (60) - flu_low (60) - flu_high (60) - mean_low (60) - mean_high (60) - fm_low (43) - fm_high (43) - sfm_low (40) - sfm_high (36) - All (120)
0.654582957149802
0.023380909562837383
[0.686756024723939, 0.6839433293978748, 0.6794013473157858, 0.6592332800888949, 0.6539273560663934, 0.6500451420237515, 0.6466629627057434, 0.6402458504062781, 0.628995069102021, 0.6166192096673379]
